In [106]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import os

In [143]:
# kospi_list = ["005380","035420","068270","005930","000270","373220","207940","000660","006400","051910"]
df=pd.DataFrame()

kospi_list = {"005380":'현대차',"035420":"NAVER","068270":'셀트리온',"005930":'삼성전자',"000270":'기아',"373220":'LG에너지솔루션',"207940":'삼성바이오로직스',"000660":'sk하이닉스',"006400":'삼성SDI',"051910":"LG화학"}
for code in kospi_list:
    temp_page = 1000
    end_url = 'https://finance.naver.com/research/company_list.naver?keyword=&brokerCode=&writeFromDate=&writeToDate=&searchType=itemCode&itemName='+kospi_list[code]+'&itemCode='+ code + '&page=1000'
    response = requests.get(end_url)
    soup = BeautifulSoup(response.text,"html.parser")
    end_page = soup.select_one('td.on a').text

    for i in range(1,int(end_page)+1):
        page = i
        page_url = 'https://finance.naver.com/research/company_list.naver?keyword=&brokerCode=&writeFromDate=&writeToDate=&searchType=itemCode&itemName='+kospi_list[code]+'&itemCode='+ code + '&page='+str(page)
        page_response = requests.get(page_url)
        page_soup = BeautifulSoup(page_response.text,"html.parser")

        for i in range(1,len(soup.select('a[href*="company_read"]'))):
            try:
                temp = page_soup.select('a[href*="company_read"]')[i]
                parm = temp['href']
                main_url = 'https://finance.naver.com/research/'+parm
                main_response = requests.get(main_url)
                main_soup = BeautifulSoup(main_response.text,"html.parser")


                result=main_soup.select_one('.view_sbj').text.split('\n')

                company=kospi_list[code]
                title = result[2].strip()
                firm = result[3].strip().split('|')[0]
                date = result[3].strip().split('|')[1]
                money = main_soup.select_one('.money').text
                coment = main_soup.select_one('.coment').text
                pdf_link = main_soup.select_one('.view_report a')['href']

                new_data = pd.DataFrame({
                    'company' : [company],
                    'title' : [title],
                    'firm' : [firm],
                    'date' : [date],
                    'money' : [money],
                    'coment' : [coment],
                    'pdf_link' : [pdf_link]

                })
                df = pd.concat([df, new_data])
            except:
                print('오류링크:',main_url)


오류링크: https://finance.naver.com/research/company_read.naver?nid=32843&page=8&searchType=itemCode&itemCode=000660


In [144]:
df.to_csv('./../../Data/kospi_report.csv',encoding='utf-8-sig')

In [ ]:
# pdf 다운로드 함수
def download(download_url, file_name):
    with open(file_name, "wb") as file:  
        response = requests.get(download_url)
        file.write(response.content)

In [ ]:
if not os.path.isdir('./../../Data/report_pdf'):
    os.mkdir('./../../Data/report_pdf')
os.chdir('./../../Data/report_pdf')